# 13 NetCDF and `xarray`

In this lesson, we will get acquainted with a popuar format for working with multidimensional datasets called NetCDF and the Python package `xarray` which is based on NetCDF. 


`xarray` is a Python package that augments NumPy by adding labeled dimensions, coordinates, and attributes.

`xarray` is based on the NetCDF data model and it's a great tool to open, process, and create datasets in this data format.

## `xarray.DataArray`

The `xarray.DataArray` is the primary structure of the `xarray` package. It is an n-dimensional array with labeled dimensions.

We can think of an `xarray.DataArray` as representing **a single variable** in the NetCDF data format: it has the variables values, dimensions, coordinates, and attributes.

We will create a small `xarray.DataArray` from scratch to exemplify this. 

In [1]:
# Import packages

import os
import numpy as np
import pandas as pd
import xarray as xr

### Variable values

The underlying data in the `xarray.DataArray` is a `numpy.array` that holds the variable values. 

In [6]:
# Values of a single variable at each point of the coords 
temp_data = np.array([np.zeros((5,5)),
                     np.ones((5,5)),
                     np.ones((5,5))*2]).astype(int)

In [7]:
temp_data

array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])

### Dimensions and Coordinates

To specify the dimensions of our upcoming `xarray.DataArray`, we must examine how we've constructed the `numpy.array` holding the temperature data. 
The first dimension is time, the second is latitude, and longitude the third. 

From our exercises, we can also see that the coordinates (values of each dimension) are:

- time coordinates are 2022-09-01, 2022-09-02, 2022-09-03
- latitude coordinates are 70, 60, 50, 40, 30 (notice decreasing order)
- longitude coordinates are 60, 70, 80, 90, 100 (notice increasing order)

We add the dimensions as a tuple of strings and coordinates as a dictionary:

In [9]:
# Names of the dimensions in the required order

# Create coordinates to use for indexing along each dimension 
coords = {'time':pd.date_range("2022-09-01", "2022-09-03"),
          'lat':np.arange(70, 20, -10),
          'lon':np.arange(60, 110, 10)}  

#### Attributes

Next, we add the attributes (metadata) for our temperature data as a dictionary:

In [11]:
# Attributes (metadata) of the data array 
attrs = { 'title':'temperature across weather stations',
          'standard_name':'air_temperature',
          'units':'degree_c'}

#### Putting it all together

Finally, we put all these pieces together (data, dimensions, coordinates, and attributes) to create an `xarray.DataArray`:

In [19]:


# Initialize xarray.DataArray
temp = xr.DataArray(data = temp_data, 
                    coords = coords,
                    attrs = attrs)
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)> Size: 600B
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 40B 70 60 50 40 30
  * lon      (lon) int64 40B 60 70 80 90 100
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c

We can also update the variable’s attributes after creating the object. 
Notice that each of the coordinates is also an `xarray.DataArray`, so we can add attributes to them.

In [20]:
# Update attributes
temp.attrs['description'] = 'Simple example of an xarray.DataArray'

# Add attributes to coordinates 
temp.time.attrs = {'description':'date of measurement'}

temp.lat.attrs['standard_name']= 'grid_latitude'
temp.lat.attrs['units'] = 'degree_N'

temp.lon.attrs['standard_name']= 'grid_longitude'
temp.lon.attrs['units'] = 'degree_E'
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)> Size: 600B
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 40B 70 60 50 40 30
  * lon      (lon) int64 40B 60 70 80 90 100
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

## Reduction

`xarray` has several methods to reduce an xr.DataArray along a number of dimensions.
For example, calculate the average temperature at each weather station over time and obtain a new xr.DataArray

In [22]:
avg_temp = temp.mean(dim='time')
avg_temp.attrs = {'title': 'Average temperature across three days'}

## `xarray.DataSet`

An `xarray.DataSet` is an in-memory representation of a NetCDF file, with *multiple* variables.

Let's create a DataSEt for the avg temp and temp data.

In [24]:
# Dictionary with variables and attributes

data_vars = {
    'temp': temp,
    'avg_temp': avg_temp
}

attrs = {'title': 'Temperature data aat weather stations: daily and average',
        'description': 'Simply example of an xarray.DataSet'}

# Create xarray.DataSet
temp_dataset = xr.Dataset(data_vars = data_vars,
                          attrs = attrs)